In [ ]:
%%capture
!pip install datasets -U
!pip install transformers -U
!pip install torchaudio -U
!pip install librosa -U
!pip install jiwer -U
!pip install parsivar -U
!pip install num2fawords -U

In [ ]:
from datasets import load_dataset, load_metric, Audio, ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML
import re
import string
from parsivar import Normalizer
import num2fawords
from transformers import TrainingArguments, Trainer
import torch
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union
from google.colab import drive
from transformers import Wav2Vec2ForCTC, Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor, Wav2Vec2Processor
import IPython.display as ipd
import numpy as np
import json

In [ ]:
common_voice_train = load_dataset('common_voice', 'fa', split='train[:3000]')
common_voice_test = load_dataset('common_voice', 'fa', split='test[:1000]')

Generating train split:   0%|          | 0/7593 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5213 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5213 [00:00<?, ? examples/s]

Generating other split:   0%|          | 0/22510 [00:00<?, ? examples/s]

Generating validated split:   0%|          | 0/251659 [00:00<?, ? examples/s]

Generating invalidated split:   0%|          | 0/11698 [00:00<?, ? examples/s]

Dataset common_voice downloaded and prepared to /root/.cache/huggingface/datasets/common_voice/fa/6.1.0/a1dc74461f6c839bfe1e8cf1262fd4cf24297e3fbd4087a711bd090779023a5e. Subsequent calls will reuse this data.


Reusing dataset common_voice (/root/.cache/huggingface/datasets/common_voice/fa/6.1.0/a1dc74461f6c839bfe1e8cf1262fd4cf24297e3fbd4087a711bd090779023a5e)


In [ ]:
common_voice_train = common_voice_train.remove_columns(['accent', 'age', 'client_id', 'down_votes', 'gender', 'locale', 'segment', 'up_votes'])
common_voice_test = common_voice_test.remove_columns(['accent', 'age', 'client_id', 'down_votes', 'gender', 'locale', 'segment', 'up_votes'])

In [ ]:
def show_random_elements(dataset, num_examples=10):
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset) - 1)
        while pick in picks:
            pick = random.randint(0, len(dataset) - 1)

        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(common_voice_train.remove_columns(['path', 'audio']), num_examples=10)

,sentence
0,راهنمای ما چه کسی خواهد بود؟
1,تو جذاب هستی.
2,ارسال این با پست ثبت شده چقدر می شود؟
3,داروخانه چه ساعتی باز می کند؟
4,می توانی با من با شماره یک دو سه چهار پنج تماس بگیری.
5,نه ، تموم نيست تا وقتي
6,می توانم این رقص را داشته باشم، لطفا؟
7,ده و پانزده دقیقه.
8,.توئي ، يه لحظه صبر کن
9,ما جداگانه پرداخت می کنیم.


In [ ]:
def remove_special_characters(batch):
    chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'
    batch['sentence'] = re.sub(chars_to_ignore_regex, '', batch['sentence']).lower() + ' '
    return batch

In [ ]:
common_voice_train = common_voice_train.map(remove_special_characters)
common_voice_test = common_voice_test.map(remove_special_characters)

  0%|          | 0/3000 [00:00<?, ?ex/s]

  0%|          | 0/1000 [00:00<?, ?ex/s]

In [ ]:
_normalizer = Normalizer()

def multiple_replace(text, chars_to_mapping):
    pattern = '|'.join(map(re.escape, chars_to_mapping.keys()))
    return re.sub(pattern, lambda m: chars_to_mapping[m.group()], str(text))

def remove_special_characters(text, chars_to_ignore_regex):
    text = re.sub(chars_to_ignore_regex, '', text).lower() + ' '
    return text

chars_to_ignore = [
    ",", "?", ".", "!", "-", ";", ":", '""', "%", "'", '"', "�",
    "#", "!", "؟", "?", "«", "»", "،", "(", ")", "؛", "'ٔ", "٬",'ٔ', ",", "?", 
    ".", "!", "-", ";", ":",'"',"“", "%", "‘", "”", "�", "–", "…", "_", "”", '“', '„',
    'ā', 'š',
]

chars_to_ignore = chars_to_ignore + list(string.ascii_lowercase + string.digits)

chars_to_mapping = {
    'ك': 'ک', 'دِ': 'د', 'بِ': 'ب', 'زِ': 'ز', 'ذِ': 'ذ', 'شِ': 'ش', 'سِ': 'س', 'ى': 'ی',
    'ي': 'ی', 'أ': 'ا', 'ؤ': 'و', "ے": "ی", "ۀ": "ه", "ﭘ": "پ", "ﮐ": "ک", "ﯽ": "ی",
    "ﺎ": "ا", "ﺑ": "ب", "ﺘ": "ت", "ﺧ": "خ", "ﺩ": "د", "ﺱ": "س", "ﻀ": "ض", "ﻌ": "ع",
    "ﻟ": "ل", "ﻡ": "م", "ﻢ": "م", "ﻪ": "ه", "ﻮ": "و", 'ﺍ': "ا", 'ة': "ه",
    'ﯾ': "ی", 'ﯿ': "ی", 'ﺒ': "ب", 'ﺖ': "ت", 'ﺪ': "د", 'ﺮ': "ر", 'ﺴ': "س", 'ﺷ': "ش",
    'ﺸ': "ش", 'ﻋ': "ع", 'ﻤ': "م", 'ﻥ': "ن", 'ﻧ': "ن", 'ﻭ': "و", 'ﺭ': "ر", "ﮔ": "گ", 
    "ها": "  ها",
    "۱۴ام": "۱۴ ام",
    "a": " ای ", "b": " بی ", "c": " سی ", "d": " دی ", "e": " ایی ", "f": " اف ",
    "g": " جی ", "h": " اچ ", "i": " آی ", "j": " جی ", "k": " کی ", "l": " ال ",
    "m": " ام ", "n": " ان ", "o": " او ", "p": " پی ", "q": " کیو ", "r": " آر ",
    "s": " اس ", "t": " تی ", "u": " یو ", "v": " وی ", "w": " دبلیو ", "x": " اکس ",
    "y": " وای ", "z": " زد ",
    "\u200c": " ", "\u200d": " ", "\u200e": " ", "\u200f": " ", "\ufeff": " ",
}

chars_to_ignore = chars_to_ignore + list(string.ascii_lowercase + string.digits)

def normalizer_main(batch, chars_to_ignore=chars_to_ignore, chars_to_mapping=chars_to_mapping):
    chars_to_ignore_regex = f"""[{''.join(chars_to_ignore)}]"""

    text = batch['sentence'].lower().strip()
    text = _normalizer.normalize(text)
    text = multiple_replace(text, chars_to_mapping)
    text = remove_special_characters(text, chars_to_ignore_regex)
    text = re.sub(' +', ' ', text)
    
    _text = []

    for word in text.split():
        try:
            word = int(word)
            _text.append(words(word))
        except:
            _text.append(word)
            
    text = ' '.join(_text) + ' '
    text = text.strip()

    if not len(text) > 0:
        return None
    
    batch['sentence'] = text + ' '
    
    return batch

In [ ]:
common_voice_train = common_voice_train.map(normalizer_main, fn_kwargs={'chars_to_ignore': chars_to_ignore, 'chars_to_mapping': chars_to_mapping})
common_voice_test = common_voice_test.map(normalizer_main, fn_kwargs={'chars_to_ignore': chars_to_ignore, 'chars_to_mapping': chars_to_mapping})

  0%|          | 0/3000 [00:00<?, ?ex/s]

  0%|          | 0/1000 [00:00<?, ?ex/s]

In [ ]:
show_random_elements(common_voice_train.remove_columns(['path', 'audio']))

,sentence
0,او بوی قهوه را دوست دارد
1,امکان دارد یک چراغ قوه به من قرض بدهید
2,هفتاد و یک هفتاد و دو هفتاد و سه
3,تا چه ساعتی
4,نشنیدی صدای اسفندیار رو
5,آیا می توانید دندان های مصنوعی مرا درست کنید
6,خط هوایی من لوفتانزا است
7,امتیاز این حفره چند است
8,تو این جا چی کار میکنی
9,باشه دارم میام بیرون


In [ ]:
def extract_all_chars(batch):
    all_text = ' '.join(batch['sentence'])
    vocab = list(set(all_text))
    return {'vocab': [vocab], 'all_text': [all_text]}

In [ ]:
vocab_train = common_voice_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_train.column_names)
vocab_test = common_voice_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_test.column_names)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
vocab_list = list(set(vocab_train['vocab'][0]) | set(vocab_test['vocab'][0]))

In [ ]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{' ': 2,
 'آ': 4,
 'ئ': 1,
 'ا': 16,
 'ب': 20,
 'ت': 8,
 'ث': 7,
 'ج': 0,
 'ح': 14,
 'خ': 25,
 'د': 10,
 'ذ': 6,
 'ر': 17,
 'ز': 12,
 'س': 13,
 'ش': 22,
 'ص': 32,
 'ض': 15,
 'ط': 11,
 'ظ': 31,
 'ع': 9,
 'غ': 34,
 'ف': 27,
 'ق': 3,
 'ل': 29,
 'م': 21,
 'ن': 19,
 'ه': 28,
 'و': 26,
 'پ': 30,
 'چ': 18,
 'ژ': 5,
 'ک': 33,
 'گ': 24,
 'ی': 23}

In [ ]:
vocab_dict['|'] = vocab_dict[' ']
del vocab_dict[' ']

In [ ]:
vocab_dict['[UNK]'] = len(vocab_dict)
vocab_dict['[PAD]'] = len(vocab_dict)
len(vocab_dict)

37

In [ ]:
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [ ]:
tokenizer = Wav2Vec2CTCTokenizer('./vocab.json', unk_token='[UNK]', pad_token='[PAD]', word_delimiter_token='|')

In [ ]:
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [ ]:
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [ ]:
common_voice_train[0]['path']

'/root/.cache/huggingface/datasets/downloads/extracted/3b0e0ce56990e1c035c009dd3032760da75614498aab0a4090ea91c8d4e8a843/cv-corpus-6.1-2020-12-11/fa/clips/common_voice_fa_18202376.mp3'

In [ ]:
common_voice_train[0]['audio']

{'array': array([ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
        -1.5437603e-05, -9.5367432e-06, -4.7087669e-06], dtype=float32),
 'path': '/root/.cache/huggingface/datasets/downloads/extracted/3b0e0ce56990e1c035c009dd3032760da75614498aab0a4090ea91c8d4e8a843/cv-corpus-6.1-2020-12-11/fa/clips/common_voice_fa_18202376.mp3',
 'sampling_rate': 48000}

In [ ]:
common_voice_train = common_voice_train.cast_column('audio', Audio(sampling_rate=16_000))
common_voice_test = common_voice_test.cast_column('audio', Audio(sampling_rate=16_000))

In [ ]:
common_voice_train[0]['audio']

{'array': array([ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
        -1.0061510e-04, -9.2824957e-05, -1.6028429e-05], dtype=float32),
 'path': '/root/.cache/huggingface/datasets/downloads/extracted/3b0e0ce56990e1c035c009dd3032760da75614498aab0a4090ea91c8d4e8a843/cv-corpus-6.1-2020-12-11/fa/clips/common_voice_fa_18202376.mp3',
 'sampling_rate': 16000}

In [ ]:
rand_int = random.randint(0, len(common_voice_train)-1)

ipd.Audio(data=common_voice_train[rand_int]['audio']['array'], autoplay=True, rate=16000)

In [ ]:
rand_int = random.randint(0, len(common_voice_train)-1)

print('Target text:', common_voice_train[rand_int]['sentence'])
print('Input array shape:', common_voice_train[rand_int]['audio']['array'].shape)
print('Sampling rate:', common_voice_train[rand_int]['audio']['sampling_rate'])

Target text: چندتا می خواهید 
Input array shape: (44160,)
Sampling rate: 16000


In [ ]:
def prepare_dataset(batch):
    audio = batch['audio']

    batch['input_values'] = processor(audio['array'], sampling_rate=audio['sampling_rate']).input_values[0]
    
    with processor.as_target_processor():
        batch['labels'] = processor(batch['sentence']).input_ids

    return batch

In [ ]:
common_voice_train = common_voice_train.map(prepare_dataset, remove_columns=common_voice_train.column_names, num_proc=4)
common_voice_test = common_voice_test.map(prepare_dataset, remove_columns=common_voice_test.column_names, num_proc=4)

#1:   0%|          | 0/750 [00:00<?, ?ex/s]

#0:   0%|          | 0/750 [00:00<?, ?ex/s]

#3:   0%|          | 0/750 [00:00<?, ?ex/s]

#2:   0%|          | 0/750 [00:00<?, ?ex/s]

#0:   0%|          | 0/250 [00:00<?, ?ex/s]

#1:   0%|          | 0/250 [00:00<?, ?ex/s]

#2:   0%|          | 0/250 [00:00<?, ?ex/s]

#3:   0%|          | 0/250 [00:00<?, ?ex/s]

In [ ]:
@dataclass
class DataCollatorCTCWithPadding:
    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{'input_values': feature['input_values']} for feature in features]
        label_features = [{'input_ids': feature['labels']} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors='pt',
        )

        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors='pt',
            )

        labels = labels_batch['input_ids'].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch['labels'] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
wer_metric = load_metric('wer')

In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)

    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    if isinstance(label_str, list):
        if isinstance(pred_str, list) and len(pred_str) == len(label_str):
            for index in random.sample(range(len(label_str)), 3):
                print(f'reference: {label_str[index]}')
                print(f'predicted: {pred_str[index]}')
        else:
            for index in random.sample(range(len(label_str)), 3):
                print(f'reference: {label_str[index]}')
                print(f'predicted: {pred_str}')

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {'wer': wer}

In [ ]:
model = Wav2Vec2ForCTC.from_pretrained(
    'facebook/wav2vec2-large-xlsr-53', 
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    ctc_loss_reduction='mean', 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

Downloading:   0%|          | 0.00/1.73k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-large-xlsr-53 were not used when initializing Wav2Vec2ForCTC: ['project_hid.weight', 'project_q.bias', 'project_q.weight', 'quantizer.weight_proj.bias', 'project_hid.bias', 'quantizer.codevectors', 'quantizer.weight_proj.weight']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.weight', 'lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to u

In [ ]:
model.freeze_feature_extractor()

/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1677: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5.Please use the equivalent `freeze_feature_encoder` method instead.
  FutureWarning,


In [ ]:
model.gradient_checkpointing_enable()

In [ ]:
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/wav2vec2-demo-farsi',
    group_by_length=True,
    per_device_train_batch_size=10,
    per_device_eval_batch_size=10,
    gradient_accumulation_steps=2,
    evaluation_strategy='steps',
    num_train_epochs=20,
    fp16=True,
    save_steps=3000,
    eval_steps=100,
    logging_steps=10,
    learning_rate=3e-4,
    warmup_steps=500,
    save_total_limit=1,
)

Now, all instances can be passed to Trainer and we are ready to start training!

In [ ]:
trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=common_voice_train,
    eval_dataset=common_voice_test,
    tokenizer=processor.feature_extractor,
)

Using amp half precision backend


In [ ]:
fhfhg fghs bvbnvb dgfg fgh
dfhgbdfh dg dhfg dfgdfg ghjgj
dfgfd fdhgv d dfhs fgh
fgj shfsf
dg djgh gkbn fkbn 
dfkghjdhgdfjvjkbhfgj jnv hdfh fghfgd
fb hg dvbmdn fdvmdfvb
gkfb vsdvsd jsv vbn
fbnh fbfhb jfb mnb fb
wdhfg dhkbvdv dkhv 

In [ ]:
train_result = trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 3000
  Num Epochs = 20
  Instantaneous batch size per device = 10
  Total train batch size (w. parallel, distributed & accumulation) = 20
  Gradient Accumulation steps = 2
  Total optimization steps = 3000


Step,Training Loss,Validation Loss,Wer
100,7.242900,4.669602,0.998127
200,3.013300,2.991852,0.998127
300,3.000100,2.992892,0.998127
400,2.935100,2.911473,0.998127
500,2.692300,2.757064,0.998010
600,1.386700,1.208800,0.925653
700,0.931700,0.876365,0.797096
800,0.701200,0.839133,0.772158
900,0.744500,0.789213,0.754361
1000,0.552200,0.748968,0.712797


***** Running Evaluation *****
  Num examples = 1000
  Batch size = 10


reference: هنگامی که شرکت ها همپیمانی را به افرادشان آموزش میدهند برنامه های جامع و متنوع قویتر می شوند
predicted: ج
reference: متاسفانه بازار کتاب کودک پر است از این دست مفاهیم ازارنده
predicted: ج
reference: فکر می کنم اینطور باشه
predicted: ج


***** Running Evaluation *****
  Num examples = 1000
  Batch size = 10


...

***** Running Evaluation *****
  Num examples = 1000
  Batch size = 10


reference: وای اونجا رو نگاه کن
predicted: وای اونجا را نگاه کن  ج
reference: محیط های کاری مدرنی که به توافق های تجاری زمان ما خیانت کردند
predicted: معیت های کاری مدرنی که به توافق های تجاری  زمان ما خیانت کردان ج
reference: وقتی به چهره عموم نگاه کنی میفهمی که چه جورآدمیه
predicted: وقتو ه چهره عمون نگاه کنی می فهمیکه چه جورهدمی   ج


***** Running Evaluation *****
  Num examples = 1000
  Batch size = 10


reference: منم دو روزه که همینو میگم این که چیز تازه ای نیست
predicted: من امندو روسکه هوینومیگم ایدکچ استتاظهی نیست  ج
reference: یه شوهر داریم بهش میگم یکی از اخلاقای خوب من رو بگو میگه اگه بگم پررو میشی
predicted: یه شوهر داریم بهشم میگهم یکی از اخلاقای خوب من رو بگو میگه اگر بگم پررو میشید ج
reference: چقد درهم وبرهم شد معلوم نیست چی به چیه
predicted: چقدر در ها مبرحم شد معلوم نیست چی بچیه  ج


KeyboardInterrupt: ignored

In [ ]:
#metrics = train_result.metrics
#max_train_samples = len(_common_voice_train)
#metrics['train_samples'] = min(max_train_samples, len(_common_voice_train))

trainer.save_model()

#trainer.log_metrics('train', metrics)
#trainer.save_metrics('train', metrics)
trainer.save_state()

Saving model checkpoint to /content/drive/MyDrive/wav2vec2-demo-farsi
Configuration saved in /content/drive/MyDrive/wav2vec2-demo-farsi/config.json
Model weights saved in /content/drive/MyDrive/wav2vec2-demo-farsi/pytorch_model.bin
Feature extractor saved in /content/drive/MyDrive/wav2vec2-demo-farsi/preprocessor_config.json


In [ ]:
tokenizer.save_pretrained('/content/drive/MyDrive/wav2vec2-demo-farsi')

tokenizer config file saved in /content/drive/MyDrive/wav2vec2-demo-farsi/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/wav2vec2-demo-farsi/special_tokens_map.json


('/content/drive/MyDrive/wav2vec2-demo-farsi/tokenizer_config.json',
 '/content/drive/MyDrive/wav2vec2-demo-farsi/special_tokens_map.json',
 '/content/drive/MyDrive/wav2vec2-demo-farsi/vocab.json',
 '/content/drive/MyDrive/wav2vec2-demo-farsi/added_tokens.json')